# import packages

In [2]:
# Mount Google drive 
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [1]:
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
# Text representation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# ML Algo
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

# Evaluvation
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report

from time import time

In [2]:
import string
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer

In [3]:
def plotConfusionMatrixHeatmap(input_df: pd.core.frame.DataFrame, model_name: str, figsize=(20, 18)):
    """
    Return the results of a multiclass classification algorithms as a heatmap
    based on a confusion matrix.        
    """
    # Heatmap of the results
    plt.figure(figsize=figsize)
    sns.heatmap(input_df, annot=True, fmt='d', cmap='Reds')
    plt.ylabel('True', fontweight='bold')
    plt.xlabel('Predicted', fontweight='bold')
    plt.title(f'Confusion Matrix - {model_name}', size=14, fontweight='bold')
    plt.show()

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dines\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dines\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dines\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Load Training file

In [5]:
data_path = '../../data/train.csv'

In [6]:
df = pd.read_csv(data_path)

In [7]:
df.shape

(405735, 4)

In [8]:
df.columns

Index(['Unnamed: 0', 'index', 'full_text', 'label'], dtype='object')

In [9]:
df.head()

,Unnamed: 0,index,full_text,label
0,0,1,Incorrect information on your report-Informati...,Credit reporting
1,1,2,"Over the past 2 weeks, I have been receiving e...",Debt collection
2,2,3,Pioneer has committed several federal violatio...,Debt collection
3,3,8,"Previously, on XX/XX/XXXX, XX/XX/XXXX, and XX/...",Credit reporting
4,4,11,Closing your account-Company closed your account,Credit card or prepaid card


In [10]:
df.drop(columns=['Unnamed: 0', 'index'], inplace=True)

In [11]:
df.columns

Index(['full_text', 'label'], dtype='object')

In [12]:
df.head()

,full_text,label
0,Incorrect information on your report-Informati...,Credit reporting
1,"Over the past 2 weeks, I have been receiving e...",Debt collection
2,Pioneer has committed several federal violatio...,Debt collection
3,"Previously, on XX/XX/XXXX, XX/XX/XXXX, and XX/...",Credit reporting
4,Closing your account-Company closed your account,Credit card or prepaid card


In [13]:
df['label'].value_counts()

Credit reporting                 144663
Debt collection                   86653
Mortgage                          49406
Credit card or prepaid card       45740
Loans                             40921
Account service                   29044
Money transfer, VC and Others      9308
Name: label, dtype: int64

In [14]:
# Class distribution
df['label'].value_counts()/df.shape[0] * 100

Credit reporting                 35.654553
Debt collection                  21.357043
Mortgage                         12.176914
Credit card or prepaid card      11.273368
Loans                            10.085647
Account service                   7.158367
Money transfer, VC and Others     2.294108
Name: label, dtype: float64

# Pre Processing

In [15]:
def clean_text(doc):
    """
      1. Converting all text into lower case
      2. Removing classified words like xxx
      3. Remove stop words
      4. remove punctuation
      5. remove digits
      6. Wordnet lemmatizer
      7. Custom regex for further cleaning
      """
    # Set stop word as english
    stop_word = set(stopwords.words('english'))
    
    # Tokenize the sentence and make all character lower case
    doc = [x.lower() for x in word_tokenize(doc)]
    
    # Remove classified texts
    doc = [x for x in doc if x.lower() != 'xxxx' and x.lower() != 'xx' 
           and x.lower() != 'xx/xx/xxxx' and x.lower() != 'xxxx/xxxx/xxxx']
    
    # Remove stop words
    doc = [x for x in doc if x not in stop_word]
    
    # Remove Punctuation
    doc = [x for x in doc if x not in string.punctuation]
    
    # Remove Digits
    doc = [x for x in doc if not x.isdigit()]
    
    # Set NLTK Wordnet lemmatizer and lemmatize the sentence
    lemmatizer = WordNetLemmatizer()
    doc = " ".join([lemmatizer.lemmatize(word) for word in doc])

    # Regular expression to remove unwanted chars
    pattern_1 = re.compile(r"(\W)|(\d)")
    pattern_2 = re.compile(r"\s\s+")

    doc = pattern_1.sub(" ", doc)
    doc = pattern_2.sub("", doc)
    
    return doc

In [16]:
df['text_processed'] = df.apply(lambda row : clean_text(row['full_text']), axis = 1)

In [17]:
df.head()

,full_text,label,text_processed
0,Incorrect information on your report-Informati...,Credit reporting,incorrect information report information belon...
1,"Over the past 2 weeks, I have been receiving e...",Debt collection,past week receiving excessive amount telephone...
2,Pioneer has committed several federal violatio...,Debt collection,pioneer committed several federal violation pr...
3,"Previously, on XX/XX/XXXX, XX/XX/XXXX, and XX/...",Credit reporting,previously requested experian send copy verifi...
4,Closing your account-Company closed your account,Credit card or prepaid card,closing account company closed account


# Encoding and Modeling

In [18]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [19]:
df['label_id'] = label_encoder.fit_transform(df['label'])

In [20]:
# Put the label category into dict for future use
label_map = df.set_index('label_id').to_dict()['label']
label_map

{2: 'Credit reporting',
 3: 'Debt collection',
 1: 'Credit card or prepaid card',
 0: 'Account service',
 6: 'Mortgage',
 4: 'Loans',
 5: 'Money transfer, VC and Others'}

In [21]:
key_to_label_name = [x[1] for x in sorted(label_map.items())]

In [22]:
df.head()

,full_text,label,text_processed,label_id
0,Incorrect information on your report-Informati...,Credit reporting,incorrect information report information belon...,2
1,"Over the past 2 weeks, I have been receiving e...",Debt collection,past week receiving excessive amount telephone...,3
2,Pioneer has committed several federal violatio...,Debt collection,pioneer committed several federal violation pr...,3
3,"Previously, on XX/XX/XXXX, XX/XX/XXXX, and XX/...",Credit reporting,previously requested experian send copy verifi...,2
4,Closing your account-Company closed your account,Credit card or prepaid card,closing account company closed account,1


In [23]:
X = df.text_processed
y = df.label_id

In [24]:
print(X.shape, y.shape)

(405735,) (405735,)


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [26]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(304301,) (304301,)
(101434,) (101434,)


# Multinimial Naive bayes

In [27]:
# Parameter values to test
param_grid = {
    'TfIdf__max_features' : [10000, 20000, 30000],
    'TfIdf__ngram_range' : [(1,1),(1,2),(2,2)],
    'TfIdf__use_idf' : [True],
    'MultinomialNB__alpha' : [0.01, 0.02, 0.05]
    }

In [28]:
# Creating pipeline for Naive Bayes Model
pipeline_mnb = Pipeline(steps = [('TfIdf', TfidfVectorizer()),
                              ('MultinomialNB', MultinomialNB())])

In [29]:
# Apply pipeline into the Gridserach
grid_search_mnb = GridSearchCV(pipeline_mnb, param_grid, cv=5, verbose=1, n_jobs=10)

In [ ]:
grid_search_mnb.fit(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed: 24.1min


In [ ]:
print(grid_search_mnb.best_params_)

In [ ]:
print(grid_search_mnb.best_estimator_)

In [ ]:
grid_search_mnb.score(X_test, y_test)

In [ ]:
y_predicted = grid_search_mnb.predict(X_test)

In [ ]:
classification_report_mnb = classification_report(y_test, y_predicted)
print(classification_report_mnb)

In [ ]:
conf_matrix_df = pd.DataFrame(data=confusion_matrix(y_test, y_predicted),
                              index=key_to_label_name, columns=key_to_label_name)

plotConfusionMatrixHeatmap(conf_matrix_df, model_name='Multinomial Naive bayes', figsize=(12, 10))

## Save the model

In [ ]:
import joblib

In [ ]:
joblib.dump(grid_search_mnb.best_estimator_, 'drive/MyDrive/Springboard/data/models/multinomial_naive_bayes_v3.pkl', compress = 1)

# Logistic Regression

In [ ]:
param_grid_lr = {
    'TfIdf__max_features' : [10000, 20000, 30000],
    'TfIdf__ngram_range' : [(1,1),(1,2),(2,2)],
    'TfIdf__use_idf' : [True]
    }

In [ ]:
# Creating pipeline for Logistice Regression model
pipeline_lr = Pipeline(steps = [('TfIdf', TfidfVectorizer()),
                              ('LogisticRegression', LogisticRegression(class_weight="balanced"))])

In [ ]:
grid_search_lr = GridSearchCV(pipeline_lr, param_grid_lr, cv=5, verbose=1, n_jobs=10)

In [ ]:
grid_search_lr.fit(X_train, y_train)

In [ ]:
print(grid_search_lr.best_params_)

In [ ]:
print(grid_search_lr.best_estimator_)

In [ ]:
grid_search_lr.score(X_test, y_test)

In [ ]:
y_predicted = grid_search_lr.predict(X_test)

In [ ]:
classification_report_lr = classification_report(y_test, y_predicted)
print(classification_report_lr)

In [ ]:
conf_matrix_df = pd.DataFrame(data=confusion_matrix(y_test, y_predicted),
                              index=key_to_label_name, columns=key_to_label_name)

plotConfusionMatrixHeatmap(conf_matrix_df, model_name='Logistic Regression', figsize=(12, 10))

In [ ]:
joblib.dump(grid_search_lr.best_estimator_, 'drive/MyDrive/Springboard/data/models/logistic_regression_v3.pkl', compress = 1)

# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = TfidfVectorizer(min_df=3, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))

In [ ]:
pipeline_rf = Pipeline(steps = [('countvectorizer', vectorizer),
                              ('clf', RandomForestClassifier())])

In [ ]:
model = pipeline_rf.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred_prob = model.predict_proba(X_test)

In [ ]:
lr_probs = y_pred_prob[:,1]

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
conf_matrix_df = pd.DataFrame(data=confusion_matrix(y_test, y_pred),index=key_to_label_name,
                              columns=key_to_label_name)

In [ ]:
classification_rep = classification_report(y_test, y_pred,target_names=key_to_label_name)
print(classification_rep)

In [ ]:
plotConfusionMatrixHeatmap(conf_matrix_df, model_name='Random forest', figsize=(12, 10))

In [ ]:
joblib.dump(model, 'drive/MyDrive/Springboard/data/models/random_forest_v3.pkl', compress = 1)

# Doc2Vec with Logistic Regression

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
#prepare training data in doc2vec format:
train_doc2vec = [TaggedDocument((d), tags=[str(i)]) for i, d in enumerate(X_train)]

In [ ]:
#Train a doc2vec model to learn
model = Doc2Vec(vector_size=50, alpha=0.025, min_count=5, dm =1, epochs=100)
model.build_vocab(train_doc2vec)
model.train(train_doc2vec, total_examples=model.corpus_count, epochs=model.epochs)
model.save("drive/MyDrive/Springboard/data/models/d2v-v3.model")
print("Model Saved")

In [ ]:
#Infer the feature representation for training and test data using the trained model
model= Doc2Vec.load("drive/MyDrive/Springboard/data/models/d2v-v3.model")

In [ ]:
#infer in multiple steps to get a stable representation. 
train_vectors =  [model.infer_vector(list_of_tokens, steps=50) for list_of_tokens in X_train]
test_vectors = [model.infer_vector(list_of_tokens, steps=50) for list_of_tokens in X_test]

In [ ]:
clf = LogisticRegression(class_weight="balanced")
clf.fit(train_vectors, y_train)

In [ ]:
preds = clf.predict(test_vectors)

In [ ]:
print(classification_report(y_test, preds))

In [ ]:
conf_matrix_df = pd.DataFrame(data=confusion_matrix(y_test, preds),
                              index=key_to_label_name, columns=key_to_label_name)

plotConfusionMatrixHeatmap(conf_matrix_df, model_name='Doc2vec', figsize=(12, 10))

In [ ]:
joblib.dump(clf, 'drive/MyDrive/Springboard/data/models/logistic_doc2vec_v3.pkl', compress = 1)

# Deep Learning

# Loan Data Only

In [ ]:
Random Forest Classifier